## Model the NPR articles with Latent Dirichlet Allocation

  1. Run the LDA model with sklearn (http://scikit-learn.org/stable/modules/decomposition.html#latentdirichletallocation)
  2. Visualize it with pyldavis (https://pyldavis.readthedocs.io/en/latest)

In [1]:
# USE Python3
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning, module='.*/IPython/.*')
warnings.filterwarnings('ignore', category=DeprecationWarning, module='pyLDAvis')

import pyLDAvis
import pyLDAvis.sklearn

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

pyLDAvis.enable_notebook()

df = pd.read_csv('npr_articles.csv', parse_dates=['date_published'])
text = df['processed_text'].values.tolist()

## Vectorize the words

Essentially create a numeric representation of the words based on frequencies

In [2]:
max_features = 1000
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=max_features,
                                stop_words='english')
tf = tf_vectorizer.fit_transform(text)
print("ready")

ready


## Run LDA

In [5]:
n_topics = 10
lda_model = LatentDirichletAllocation(n_components=n_topics, max_iter=5,
                                      learning_method='online',
                                      learning_offset=50.,
                                      random_state=0)

lda_model.fit(tf)
pyLDAvis.sklearn.prepare(lda_model,tf, tf_vectorizer, R=20)

/home/adam/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
7      21.078897        1       1  0.100428 -0.032692
5      18.847600        1       2 -0.049768  0.038430
9      15.857898        1       3 -0.123715  0.026187
8      11.957109        1       4 -0.021344  0.076798
1       9.628463        1       5  0.065152  0.058131
3       9.050151        1       6  0.123818  0.100956
2       7.364359        1       7 -0.166731 -0.004437
0       3.505611        1       8  0.103372  0.055559
6       2.558823        1       9  0.084934 -0.247357
4       0.151090        1      10 -0.116147 -0.071575, topic_info=     Category          Freq            Term         Total  loglift  logprob
term                                                                       
592   Default   3132.000000             npr   3132.000000  20.0000  20.0000
937   Default   2411.000000           trump   2411.000000  19.0000  19.0000
669   Default   2114.000000       president   2114.000000  18.0000  18.0000
582   Default   1754.000000             new   1754.000000  17.0000  17.0000
193   Default   2103.000000       copyright   2103.000000  16.0000  16.0000
780   Default  10223.000000             say  10223.000000  15.0000  15.0000
571   Default    752.000000           music    752.000000  14.0000  14.0000
741   Default   1782.000000          report   1782.000000  13.0000  13.0000
964   Default    625.000000            vote    625.000000  12.0000  12.0000
282   Default    754.000000        election    754.000000  11.0000  11.0000
466   Default    782.000000             job    782.000000  10.0000  10.0000
355   Default    478.000000          follow    478.000000   9.0000   9.0000
168   Default    819.000000         company    819.000000   8.0000   8.0000
870   Default    663.000000         student    663.000000   7.0000   7.0000
839   Default    568.000000            song    568.000000   6.0000   6.0000
399   Default    222.000000           happy    222.000000   5.0000   5.0000
774   Default    596.000000          russia    596.000000   4.0000   4.0000
281   Default    775.000000           elect    775.000000   3.0000   3.0000
650   Default    634.000000          police    634.000000   2.0000   2.0000
147   Default    804.000000            city    804.000000   1.0000   1.0000
745    Topic1    350.651201     republicans    352.353223   1.5521  -5.2503
237    Topic1    307.957241       democrats    309.686995   1.5513  -5.3801
596    Topic1    178.803235       obamacare    180.601384   1.5469  -5.9238
739    Topic1    254.231736          repeal    257.103555   1.5457  -5.5718
384    Topic1    103.747762             gop    105.096965   1.5440  -6.4681
937    Topic1   2380.175675           trump   2411.295509   1.5439  -3.3351
442    Topic1    126.053721        incoming    128.993738   1.5338  -6.2733
155    Topic1    185.542033         clinton    190.194634   1.5321  -5.8868
20     Topic1    468.775935  administration    481.311653   1.5305  -4.9599
595    Topic1    826.385254           obama    850.738571   1.5279  -4.3930
...       ...           ...             ...           ...      ...      ...
168    Topic9    129.994403         company    819.547769   1.8244  -4.1338
889    Topic9    127.282074            talk    853.158392   1.7631  -4.1549
669    Topic9    159.650046       president   2114.732028   1.0819  -3.9283
259    Topic9     80.848605    donald_trump    573.629931   1.7062  -4.6087
281    Topic9     78.311815           elect    775.908129   1.3723  -4.6406
582    Topic9     76.430230             new   1754.187301   0.5322  -4.6649
399   Topic10     74.911842           happy    222.962321   5.4044  -1.8556
683   Topic10     41.757870        producer    154.596064   5.1861  -2.4400
908   Topic10     31.034722       this_week    194.093850   4.6618  -2.7368
896   Topic10     11.407626      television    124.286679   4.1067  -3.7376
305   Topic10      7.821147       executi

In [7]:
def get_top_words(model, feature_names, n_top_words):
    top_words = {}
    for topic_idx, topic in enumerate(model.components_):
        _top_words = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        top_words[str(topic_idx)] = _top_words
    return(top_words)

In [8]:
## get the token to topic matrix
word_topic = np.zeros((max_features,n_topics),)
print(n_topics)
lda_model.components_
for topic_idx, topic in enumerate(lda_model.components_):
    word_topic[:,topic_idx] = topic

print("token-topic matrix",word_topic.shape)

## create a matrix of the top words used to define each topic
top_words = 15
tf_feature_names = np.array(tf_vectorizer.get_feature_names())
top_words = get_top_words(lda_model,tf_feature_names,top_words)
all_top_words = np.array(list(set().union(*[v for v in top_words.values()])))

for key,vals in top_words.items():
    print(key," ".join(vals))
print("total words: %s"%len(all_top_words))

top_word_inds = [np.where(tf_feature_names == tw)[0][0] for tw in all_top_words]

20
token-topic matrix (1000, 20)
0 vote power party political government say opposition voter win change movement election anti conservative result
1 say city water people work company home make use resident community house local building npr
2 sessions hearing attorney alabama general confirmation civil senator right sen record the_united_states justice sexual tuesday
3 rate park area report increase population say high accord rise number people california bear rural
4 producer happy this_week station favorite twitter follow new television local executive facebook movie review close
5 say science human scientist change new use world way space time make earth universe look
6 job think president say make people come tax company trump thing secretary want write new
7 say trump president obama election npr elect state campaign news make people time report donald_trump
8 say report people police npr attack tell officer city kill man force case death accord
9 christmas tree car holiday word